In [3]:
import sys, os
sys.path.append(os.getcwd())
sys.path.append('../..')
import graph_parser
from imports import ilpgraph
from network import PCST, PCST_Linear
import networkx as nx
from gurobipy import *
import xlsxwriter

def __write_solution(solution:list):
    
    workbook = xlsxwriter.Workbook('Benchmarks4.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('D1', 'LinearModel', bold)
    worksheet.write('D1', 'cycleBasis', bold)
    worksheet.write('E1', 'nodeColouring', bold)
    worksheet.write('F1', 'Runtime', bold)
    worksheet.write('G1', 'Objective Value', bold)
    worksheet.write('H1', 'Constraints', bold)
    worksheet.write('I1', 'MIP Gap', bold)
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    workbook.close()
    print("Finished!")
    
    
def runBenchmark(settings:list, path:str, linesToRead:range):
    solutions = []
    lines = []
    with open(path) as f:
        formattedFile = f.read().split('\n')
        for i in linesToRead:
            lines.append(formattedFile[i])

    for i in range(len(linesToRead)):
        G, start_terminal, cost_per_meter, prob_id = graph_parser.parse_vdf(lines[i])
        Graph = ilpgraph.ILPGraph()
        Graph.setNXGraph(G)
        Graph.setEdgeVars(G.edges())
        Graph.setNodeVars(G.nodes())
        
        for setting in settings:
            takeLinear = setting[0]
            cycleBasis = setting[1]
            nodeColoring = setting[2]
            # Normal version is to be chosen
            if (takeLinear == False):
                model = PCST.createModel(Graph, start_terminal, cycleBasis, nodeColoring)
                model.params.MIPGap = 0.05
                model.Params.threads = 3
                model.params.TimeLimit = 600
                model.optimize(PCST.callback_cycle) 
            else:
                model = PCST_Linear.createModel(Graph, start_terminal,  cycleBasis, nodeColoring)
                model.Params.threads = 3
                model.params.MIPGap = 0.05
                model.params.TimeLimit = 600
                if (cycleBasis == True):
                    model.optimize(PCST.callback_cycle) 
                else:
                    model.optimize()
            
            
            solutions.append((i, len(G.edges()), len(G.nodes()), takeLinear, cycleBasis, nodeColoring, model.runTime, model.objVal, model.numConstrs, model.MIPGap))
    __write_solution(solutions)  
    print(PCST_Linear.extractSolution(Graph, model))

if __name__ == '__main__':
    linesToRead = range(0,1)
    
    #settings = [(False, True, False), (True, True, False), (True, True, False), (False, True, True), (True, False, True), (True, True, True),
    #            (False, False, False), (False, False, True)]
    
    settings = [(True, False, False)]
    path = '/mnt/data/projects/ILP/optimisation_instances_02_he_new.txt'
    
    runBenchmark(settings, path, linesToRead)
    




Using license file /opt/gurobi/gurobi.lic
Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Parameter threads unchanged
   Value: 3  Min: 0  Max: 1024  Default: 0
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 2890 rows, 2308 columns and 11997 nonzeros
Model fingerprint: 0x13a54e69
Variable types: 0 continuous, 2308 integer (1876 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+02, 5e+04]
  Bounds range     [1e+00, 4e+02]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 569 rows and 189 col

# Testing Steiner Module

In [1]:
import itertools
import sys
sys.path.append("..")
from imports import ilpgraph,readFile
from network import Steiner
import networkx as nx
from gurobipy import *


def runBenchmark(settings:list, path:str):
    solutions = []
    lines = []


    G, terminals = readFile.stp_to_networkx(path)
    
    Graph = ilpgraph.ILPGraph()
    Graph.setNXGraph(G)
    Graph.setEdgeVars(G.edges())
    Graph.setNodeVars(G.nodes())
    
    for setting in settings:
        cycleBasis = setting[0]
        nodeColoring = setting[1]
        model = Steiner.createModel(Graph, terminals, cycleBasis, nodeColoring)
        model.Params.threads = 3
        model.params.MIPGap = 0.02
        model.params.TimeLimit = 600
        if (cycleBasis):
            model.optimize(Steiner.callback_cycle) 
        else:
            model.optimize()
        solutions.append((len(G.edges()), len(G.nodes()), cycleBasis, nodeColoring, model.runTime, model.objVal,\
                          model.numConstrs, model.MIPGap))
        
    #__write_solution(solutions)         
    print(Steiner.extractSolution(Graph, model))



if __name__ == '__main__':
    
    #settings = [(False, False), (True,False), (False, True), (True, True)]
    settings = [(False, False)]
    path = '/mnt/data/projects/ILP/steiner/PUC/bip42u.stp'
    
    runBenchmark(settings, path)


Using license file /opt/gurobi/gurobi.lic
Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter MIPGap to 0.02
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 5383 rows, 6382 columns and 26492 nonzeros
Model fingerprint: 0xd187df5c
Variable types: 0 continuous, 6382 integer (6382 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 389.0000000
Presolve removed 296 rows and 1453 columns
Presolve time: 0.05s
Presolved: 5087 rows, 4929 columns, 21617 nonzeros
Variable types: 0 continuous, 4929 integer (4929 binary)

Root relaxation: obje

# Steiner Linear Implementation Test

In [1]:
import itertools
import sys
sys.path.append("..")
from imports import ilpgraph,readFile
from network import Steiner_Linear
import networkx as nx
from gurobipy import *


def runBenchmark(path:str):
    solutions = []
    lines = []


    G, terminals = readFile.stp_to_networkx(path)
    
    Graph = ilpgraph.ILPGraph()
    Graph.setNXGraph(G)
    Graph.setEdgeVars(G.edges())
    Graph.setNodeVars(G.nodes())
    
    model = Steiner_Linear.createModel(Graph, terminals)
    model.params.MIPGap = 0.02
    model.Params.threads = 3
    model.params.TimeLimit = 600
    model.optimize()
    solutions.append((len(G.edges()), len(G.nodes()), model.runTime, model.objVal,\
                      model.numConstrs, model.MIPGap))
        
    #__write_solution(solutions)         


if __name__ == '__main__':
    
    path = '/mnt/data/projects/ILP/steiner/PUC/bip42u.stp'
    runBenchmark(path)


Using license file /opt/gurobi/gurobi.lic
Changed value of parameter MIPGap to 0.02
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 18529 rows, 10364 columns and 90204 nonzeros
Model fingerprint: 0xaae93009
Variable types: 0 continuous, 10364 integer (9164 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Presolve removed 4278 rows and 248 columns
Presolve time: 0.11s
Presolved: 14251 rows, 10116 columns, 70262 nonzeros
Variable types: 0 continuous, 10116 integer (8916 binary)

Root relaxation: objective 2.318806e+02, 13664 iterations, 1.51